# Problem 1: Equity Volatility Estimation 

### GARCH(1, 1) model

Following the slides, given that $r_t = \mu + \sigma_t \epsilon_t$,

$$\sigma_t^2 = \alpha_0 + \alpha_1 a_{t-1}^2 + \beta_1 \sigma_{t-1}^2,$$
    
where $a_t = \sigma_t \epsilon_t$.

### (a) Estimate the parameters of a GARCH(1,1) model

Given a parameter set $\mathbf{\theta} = \{\theta, \sigma_0, \mu, \alpha_0, \alpha_1, \beta_1\}$ and a sequence of stock return $\{r_t\}$, we can first calculate $\{a_t\}$.

And based on $\sigma_t^2 = \alpha_0 + \alpha_1 a_{t-1}^2 + \beta_1 \sigma_{t-1}^2$ with our parameter set, we can calculate $\{ \sigma_t \}$.

Then, from class we have, the log-likelihood of GARCH(1,1) as,
$$
\ln \mathcal{L}(\sigma_0, \alpha_0, \alpha_1, \beta_1, \mu \mid r_1, r_2, \dots, r_T) = -\frac{T}{2} \ln(2 \pi) - \frac{1}{2} \sum_{i=1}^T \ln \sigma_i^2 - \frac{1}{2} \sum_{i=1}^T \left(\frac{(r_i - \mu)^2}{\sigma_i^2}\right)
$$

Download SPY and calculate daily return and store them into a pickle file.

In [1]:
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import pickle

In [2]:
spy_data = yf.download("SPY", start="2015-01-02", end="2024-10-26")
spy_data['Daily Return'] = np.log(spy_data['Adj Close'] / spy_data['Adj Close'].shift(1))
spy_data.dropna(inplace=True)

with open("spy_daily_returns.pkl", "wb") as f:
    pickle.dump(spy_data['Daily Return'], f)

print("Data saved to spy_daily_returns.pkl")

[*********************100%%**********************]  1 of 1 completed

Data saved to spy_daily_returns.pkl


In [3]:
# Load the pickle file
with open("spy_daily_returns.pkl", "rb") as f:
    spy_daily_returns = pickle.load(f)

# Convert to DataFrame if it's not already one
if not isinstance(spy_daily_returns, pd.DataFrame):
    spy_daily_returns = pd.DataFrame(spy_daily_returns)

spy_daily_returns.head()

,Daily Return
Date,
2015-01-05,-0.018225
2015-01-06,-0.009463
2015-01-07,0.012384
2015-01-08,0.017589
2015-01-09,-0.008046


In [4]:
r = spy_daily_returns.values
time_index = spy_daily_returns.index

In [5]:
def neg_likelihood_func(r, mu, var):
    val = 0.5 * np.sum((r - mu) ** 2 / var) + 0.5 * np.log(var).sum() + 0.5 * len(r) * np.log(2 * np.pi)
    return val

In [6]:
from scipy.optimize import minimize


def neg_likelihood_func_with_params(params):
    alpha0, alpha1, beta1, mu, sigma0 = params
    N = len(r)
    vars = np.zeros((N, 1))
    vars[0] = sigma0 ** 2
    a = r - mu
    squared_a = a ** 2
    for i in range(1, N):
        vars[i] = alpha0 + alpha1 * squared_a[i - 1] + beta1 * vars[i - 1]
    neg_log_val = neg_likelihood_func(r, mu, vars)
    return neg_log_val


x0 = [0.1, 0.2, 0.4, np.mean(r), np.std(r)]

# bounds = [
#     (1e-6, None),  # alpha0 > 0
#     (0, 1),  # 0 <= alpha1 <= 1
#     (0, 1),  # 0 <= beta1 <= 1
#     (-0.1, 0.1),
#     (1e-6, 0.4)
# ]

options = {
    'maxiter': 100000,
    'ftol': 1e-15,
    'disp': True
}

result1 = minimize(
    neg_likelihood_func_with_params,
    x0,
    method='Nelder-Mead',
    # bounds=bounds,
    # options=options
)

print("Optimized Parameters:", result1.x)
print("Minimum Negative Log-Likelihood:", result1.fun)

/var/folders/fb/bhld2vy11kjg5bh1hpblh9fm0000gn/T/ipykernel_30319/2615311663.py:2: RuntimeWarning: invalid value encountered in log
  val = 0.5 * np.sum((r - mu) ** 2 / var) + 0.5 * np.log(var).sum() + 0.5 * len(r) * np.log(2 * np.pi)


Optimized Parameters: [3.63036252e-06 1.83170962e-01 7.87888338e-01 7.76217661e-04
 1.02168231e-02]
Minimum Negative Log-Likelihood: -8219.0034073383


In [7]:
alpha0, alpha1, beta1, mu, sigma0 = result1.x
print(f"alpha0: {round(alpha0, 6)}")
print(f"alpha1: {round(alpha1, 6)}")
print(f"beta1: {round(beta1, 6)}")
print(f"mu: {round(mu, 6)}")

alpha0: 4e-06
alpha1: 0.183171
beta1: 0.787888
mu: 0.000776


Therefore, the optimized parameters are printed above.

(b) Estimate a GARCH(1,1) model with conditional returns having a Generalized Error Distribution.


Following the course note, we have Log-likelihood function for GED

$$
\ln \mathcal{L}(\alpha_0, \alpha_1, \beta_1, \mu, \nu | r_1, r_2, \dots, r_T) =
T \left(\ln(\nu) - \ln(\lambda) - \left(1 + \frac{1}{\nu}\right) \ln(2) - \ln\left(\Gamma\left(\frac{1}{\nu}\right)\right)\right) 
- \frac{1}{2} \sum_{i=1}^{T} \ln \sigma_i^2 - \frac{1}{2} \sum_{i=1}^{T} \left(\frac{(r_i - \mu)^2}{\lambda^2 \sigma_i^2}\right)^{\nu / 2}.
$$

By adding the extra parameter, $\nu$, we can estimate the GARCH(1,1) model with conditional returns having a Generalized Error Distribution.


In [8]:
from scipy.special import gamma


def calculate_lambda(nu):
    return (2 ** -(2 / nu) * gamma(1 / nu) / gamma(3 / nu)) ** 0.5


# Negative log-likelihood function for GED with lambda calculation
def neg_likelihood_func_GED(r, mu, var, nu):
    lambda_val = calculate_lambda(nu)
    T = len(r)
    val = (0.5 * np.sum(((r - mu) ** 2 / (lambda_val ** 2 * var)) ** (nu / 2))
           + 0.5 * np.log(var).sum()) - T * (
                      np.log(nu) - np.log(lambda_val) - (1 + 1 / nu) * np.log(2) - np.log(gamma(1 / nu)))
    return val


def neg_likelihood_func_with_params_GED(params):
    alpha0, alpha1, beta1, mu, sigma0, nu = params
    N = len(r)
    vars = np.zeros(N)
    vars[0] = sigma0 ** 2
    a = r - mu
    squared_a = a ** 2
    for i in range(1, N):
        vars[i] = alpha0 + alpha1 * squared_a[i - 1] + beta1 * vars[i - 1]
    # Calculate the negative log-likelihood using the custom GED function
    neg_log_val = neg_likelihood_func_GED(r, mu, vars, nu)
    return neg_log_val


x0 = [np.std(r), 0.2, 0.4, np.mean(r), np.std(r), 1]
# 
# bounds = [
#     (1e-6, None),  # alpha0 > 0
#     (0, 1),  # 0 <= alpha1 <= 1
#     (0, 1),  # 0 <= beta1 <= 1
#     (-0.1, 0.1),
#     (1e-6, 1),  # sigma0 > 0
#     (0.5, 10)
# ]

# options = {
#     'maxiter': 100000,
#     'ftol': 1e-10,
#     'disp': True
# }

result = minimize(
    neg_likelihood_func_with_params_GED,
    x0,
    method='Nelder-Mead',
    # options=options
)

print("Optimized Parameters:", result.x)
print("Minimum Negative Log-Likelihood:", result.fun)

/var/folders/fb/bhld2vy11kjg5bh1hpblh9fm0000gn/T/ipykernel_30319/196883460.py:12: RuntimeWarning: invalid value encountered in power
  val = (0.5 * np.sum(((r - mu) ** 2 / (lambda_val ** 2 * var)) ** (nu / 2))
/var/folders/fb/bhld2vy11kjg5bh1hpblh9fm0000gn/T/ipykernel_30319/196883460.py:13: RuntimeWarning: invalid value encountered in log
  + 0.5 * np.log(var).sum()) - T * (


Optimized Parameters: [ 1.15389837e-05 -1.22719026e-04  9.97369016e-01 -1.79459668e-02
  6.63503154e-02  3.75006564e+01]
Minimum Negative Log-Likelihood: -3579.0691825903937


In [9]:
alpha0_GED, alpha1_GED, beta1_GED, mu_GED, sigma0_GED, nu = result.x
print("Estimated Parameters for GARCH(1,1) with GED")
print(f"alpha0: {round(alpha0_GED, 6)}")
print(f"alpha1: {round(alpha1_GED, 6)}")
print(f"beta1: {round(beta1_GED, 6)}")
print(f"mu: {round(mu_GED, 6)}")
print(f"nu: {round(nu, 6)}")

Estimated Parameters for GARCH(1,1) with GED
alpha0: 1.2e-05
alpha1: -0.000123
beta1: 0.997369
mu: -0.017946
nu: 37.500656


### (c) Estimate a TGARCH(1,1) model with conditional returns having a normal distribution.

$$
\sigma_t^2 = \alpha_0 + \alpha_1 a_{t-1}^2 + \gamma_1 S_{t-1} a_{t-1}^2 + \beta_1 \sigma_{t-1}^2
$$

where

$$
S_{t-1} = \begin{cases} 
      1, & \text{if } a_{t-1} < 0; \\
      0, & \text{if } a_{t-1} \geq 0. 
   \end{cases}
$$

- Additional parameter, $ \gamma_1$ to estimate



In [10]:
from scipy.optimize import minimize


def neg_likelihood_func(r, mu, var):
    val = (0.5 * np.sum((r - mu) ** 2 / var) + 0.5 * np.log(var).sum())
    return val


def neg_likelihood_func_with_params_TGARCH(params):
    alpha0, alpha1, beta1, mu, sigma0, gamma1 = params
    N = len(r)
    vars = np.zeros((N, 1))
    vars[0] = sigma0 ** 2
    a = r - mu
    squared_a = a ** 2
    for i in range(1, N):
        S_t = a[i - 1] < 0
        vars[i] = alpha0 + alpha1 * squared_a[i - 1] + gamma1 * S_t * squared_a[i - 1] + beta1 * vars[i - 1]
    neg_log_val = neg_likelihood_func(r, mu, vars)
    return neg_log_val


x0 = [np.std(r), 0.1, 0.2, np.mean(r), np.std(r), 0.2]

# bounds = [
#     (1e-6, None),  # alpha0 > 0
#     (0, 1),  # 0 <= alpha1 <= 1
#     (0, 1),  # 0 <= beta1 <= 1
#     (-0.1, 0.1),  # no bounds for mu
#     (1e-6, 1),  # sigma0 > 0
#     (-0.5, 0.5)  # -0.5 <= gamma1 <= 0.5
# ]


result_tgarch = minimize(
    neg_likelihood_func_with_params_TGARCH,
    x0,
    method='Nelder-Mead',
    # bounds=bounds,
)

print("Optimized Parameters:", result_tgarch.x)
print("Minimum Negative Log-Likelihood:", result_tgarch.fun)

/var/folders/fb/bhld2vy11kjg5bh1hpblh9fm0000gn/T/ipykernel_30319/1858343185.py:5: RuntimeWarning: invalid value encountered in log
  val = (0.5 * np.sum((r - mu) ** 2 / var) + 0.5 * np.log(var).sum())


Optimized Parameters: [ 5.02772560e-06  3.24360784e-02  8.03099357e-01 -7.53482475e-04
  3.24523659e-02  2.62455808e-01]
Minimum Negative Log-Likelihood: -10474.892512094586


In [11]:
alpha0_TGARCH, alpha1_TGARCH, beta1_TGARCH, mu_TGARCH, sigma0_TGARCH, gamma1 = result_tgarch.x
print("Estimated Parameters for TGARCH(1,1) with Normal Distribution")
print(f"alpha0: {round(alpha0_TGARCH, 6)}")
print(f"alpha1: {round(alpha1_TGARCH, 6)}")
print(f"beta1: {round(beta1_TGARCH, 6)}")
print(f"mu: {round(mu_TGARCH, 6)}")
print(f"gamma1: {round(gamma1, 6)}")

Estimated Parameters for TGARCH(1,1) with Normal Distribution
alpha0: 5e-06
alpha1: 0.032436
beta1: 0.803099
mu: -0.000753
gamma1: 0.262456


In [12]:
pd.DataFrame(
    {
        "GARCH(1,1)": [alpha0, alpha1, beta1, mu, None],
        "GARCH(1,1) with GED": [alpha0_GED, alpha1_GED, beta1_GED, mu_GED, nu],
        "TGARCH(1,1)": [alpha0_TGARCH, alpha1_TGARCH, beta1_TGARCH, mu_TGARCH, gamma1]
    },
    index=["alpha0", "alpha1", "beta1", "mu", "nu/gamma1"]
)

,"GARCH(1,1)","GARCH(1,1) with GED","TGARCH(1,1)"
alpha0,0.000004,0.000012,0.000005
alpha1,0.183171,-0.000123,0.032436
beta1,0.787888,0.997369,0.803099
mu,0.000776,-0.017946,-0.000753
nu/gamma1,NaN,37.500656,0.262456


### (d) Compare the maximum value of the likelihood function for the models in parts (a), (b), and (c) to rank the three models.

In [13]:
def calculate_schwartz_criterion(log_likelihood, p):
    return -2 * log_likelihood + p * np.log(len(r))

In [14]:
pd.DataFrame(
    {
        "Model": ["GARCH(1,1)", "GARCH(1,1) with GED", "TGARCH(1,1)"],
        "Log-Likelihood": [-result1.fun, -result.fun, -result_tgarch.fun],
        "Schwartz Criterion": [
            calculate_schwartz_criterion(-result1.fun, 4),
            calculate_schwartz_criterion(-result.fun, 5),
            calculate_schwartz_criterion(-result_tgarch.fun, 5)
        ]
    }
)

,Model,Log-Likelihood,Schwartz Criterion
0,"GARCH(1,1)",8219.003407,-16406.758921
1,"GARCH(1,1) with GED",3579.069183,-7119.078498
2,"TGARCH(1,1)",10474.892512,-20910.725157


Conclusion:

Comparing GARCH(1,1) with GED with TGARCH(1,1) with Normal Distribution, given the same parameter number, TGARCH(1,1) has higher log-likelihood. Therefore, TGARCH(1,1) with Normal Distribution is better than GARCH(1,1) with GED.

Comparing with Schwartz Criterion, TGARCH(1,1) has the lowest value, which means TGARCH(1,1) is the best model among the three models. Then, GARCH(1,1), and GARCH(1,1) with GED.

Moreover, GARCH(1,1) with GED is producing unrealistic results and unstable to estimate. 

# Problem 2: TGARCH(1,1) Model for Option Pricing

Given prices and implied volatilitites of puts and calls on SPY options on close of Friday, October 25, 2024. The closing pricing for SPY on October 25, 2024 was 579.04. Use the TGARCH(1,1) model to price two options: 

the 588 stirke call options (the delta is approximately 0.35), and the 570 put options (the delta is approximately -0.35). Price these two options using a Monte Carlo simulation with expeirational date on Friday, November 15, 2024, which is 15 trading days from Friday, October 25, 2024. Assume the risk-free rate, $r_f = 4.86\%$ and there are no dividends before expiration.

In [24]:
# Use two months of data for implied volatility calculation
two_month_data = spy_daily_returns.iloc[-44:]
two_month_data

,Daily Return
Date,
2024-08-26,-0.002387
2024-08-27,0.001372
2024-08-28,-0.005822
2024-08-29,0.000089
2024-08-30,0.009501
2024-09-03,-0.020794
2024-09-04,-0.002049
2024-09-05,-0.002435
2024-09-06,-0.016973


Let us reestimate the parameters for TGARCH(1,1) model using the last two months of data for a more reasonable volatility prediction.

In [57]:
r = two_month_data.values

def neg_likelihood_func(r, mu, var):
    val = (0.5 * np.sum((r - mu) ** 2 / var) + 0.5 * np.log(var).sum())
    return val


def neg_likelihood_func_with_params_TGARCH(params):
    alpha0, alpha1, beta1, mu, sigma0, gamma1 = params
    N = len(r)
    vars = np.zeros((N, 1))
    vars[0] = sigma0 ** 2
    a = r - mu
    squared_a = a ** 2
    for i in range(1, N):
        S_t = a[i - 1] < 0
        vars[i] = alpha0 + alpha1 * squared_a[i - 1] + gamma1 * S_t * squared_a[i - 1] + beta1 * vars[i - 1]
    neg_log_val = neg_likelihood_func(r, mu, vars)
    return neg_log_val

# Estimate the volatility for the last month and reestimate the parameters for TGARCH
sigma = r.std()

# alpha0, alpha1, beta1, mu, sigma0, gamma1 
x0 = [np.std(r), 0.1, 0.2, np.mean(r), np.std(r), 0.]

result_tgarch = minimize(
    neg_likelihood_func_with_params_TGARCH,
    x0,
    method='Nelder-Mead'
)

print("Optimized Parameters:", result_tgarch.x)
print("Minimum Negative Log-Likelihood:", result_tgarch.fun)

Optimized Parameters: [3.13124333e-05 1.40962479e-01 2.92121870e-01 1.11028548e-03
 3.59367838e-03 6.56505306e-05]
Minimum Negative Log-Likelihood: -195.81109651466315


/var/folders/fb/bhld2vy11kjg5bh1hpblh9fm0000gn/T/ipykernel_30319/2889486759.py:4: RuntimeWarning: invalid value encountered in log
  val = (0.5 * np.sum((r - mu) ** 2 / var) + 0.5 * np.log(var).sum())


In [58]:
import math
from scipy.stats import norm


def bsm_option_price(S, K, T, r, sigma, option_type='call'):
    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == 'call':
        # Call option price
        option_price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        # Put option price
        option_price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")
    return option_price

$$r_t = \mu + \sigma_t \epsilon_t$$

$$a_t = \sigma_t \epsilon_t$$

$$
\sigma_t^2 = \alpha_0 + \alpha_1 a_{t-1}^2 + \gamma_1 S_{t-1} a_{t-1}^2 + \beta_1 \sigma_{t-1}^2
$$

where

$$
S_{t-1} = \begin{cases} 
      1, & \text{if } a_{t-1} < 0; \\
      0, & \text{if } a_{t-1} \geq 0. 
   \end{cases}
$$

Given the estimated parameter, we are first going to use the last estimated volatility to forcast the volatility each step given a simulated return, then we are finding the last final price. And calculate the simulated final payoff. Then we average out the simnulated final payoff and discounted back for the option prices. 

In [59]:
alpha0_TGARCH, alpha1_TGARCH, beta1_TGARCH, mu_TGARCH, sigma0_TGARCH, gamma1 = result_tgarch.x
alpha0_TGARCH, alpha1_TGARCH, beta1_TGARCH, mu_TGARCH, sigma0_TGARCH, gamma1

(3.131243331749098e-05,
 0.14096247941116868,
 0.2921218695188498,
 0.0011102854795353016,
 0.0035936783839148633,
 6.565053063849703e-05)

In [60]:
N = len(r)
vars_TGARCH = np.zeros((N, 1))
vars_TGARCH[0] = sigma0_TGARCH ** 2
a = r - mu_TGARCH
squared_a = a ** 2
for i in range(1, N):
    S_t = a[i - 1] < 0
    vars_TGARCH[i] = alpha0_TGARCH + alpha1_TGARCH * squared_a[i - 1] + gamma1 * S_t * squared_a[i - 1] + beta1_TGARCH * \
                     vars_TGARCH[i - 1]

sigma_TGARCH = np.sqrt(vars_TGARCH[-1])
sigma_TGARCH

array([0.00698727])

In [61]:
import numpy as np

S0 = 579.04      
K_call = 588    
K_put = 570      
T = 15 / 252     
rf = 0.0486
simulations = 10000
N = len(r)

# Monte Carlo Simulation
final_prices = np.zeros(simulations)

for sim in range(simulations):
    # Initialize variables for each simulation
    S_t = S0
    # sigma_t_squared = (im_vol/np.sqrt(252)) ** 2  # Starting variance for TGARCH
    sigma_t_squared = sigma_TGARCH ** 2  # Starting variance for TGARCH
    
    # Sequential simulation over 15 days
    for t in range(15):
        # Generate a random standard normal shock
        Z_t = np.random.normal()
        
        # Calculate the daily return based on current volatility
        daily_return = mu_TGARCH + np.sqrt(sigma_t_squared) * Z_t
        S_t = S_t * np.exp(daily_return)
        
        # Update TGARCH model with simulated return
        a_t = daily_return - mu_TGARCH
        squared_a_t = a_t ** 2
        S_indicator = int(a_t < 0)  # 1 if negative, 0 otherwise

        # Update next day's volatility using TGARCH formula
        sigma_t_squared = abs(
            alpha0_TGARCH +
            alpha1_TGARCH * squared_a_t +
            gamma1 * S_indicator * squared_a_t +
            beta1_TGARCH * sigma_t_squared
        )
    
    # Store the final simulated stock price after 15 days
    final_prices[sim] = S_t

# Calculate option payoffs
call_payoffs = np.maximum(final_prices - K_call, 0)
put_payoffs = np.maximum(K_put - final_prices, 0)

# Discount payoffs to present value
call_price = np.exp(-rf * T) * np.mean(call_payoffs)
put_price = np.exp(-rf * T) * np.mean(put_payoffs)

(a) What are the prices, and the corresponding Black-Scholes implied volatilities, for the two options?

In [62]:
print("Option Prices")
print(f"Call Option Price: {round(call_price,2)}")
print(f"Put Option Price: {round(put_price,2)}")

Option Prices
Call Option Price: 7.0
Put Option Price: 1.06


In [63]:
# Calculate Black-Scholes implied volatilities

from scipy.optimize import brentq

def call_option_implied_volatility(S, K, T, r, C, sigma_est=0.2):
    def call_option_price(sigma):
        return bsm_option_price(S, K, T, r, sigma, option_type='call') - C

    return brentq(call_option_price, 1e-6, 1)

def put_option_implied_volatility(S, K, T, r, P, sigma_est=0.2):
    def put_option_price(sigma):
        return bsm_option_price(S, K, T, r, sigma, option_type='put') - P

    return brentq(put_option_price, 1e-6, 1)

# Calculate implied volatilities
call_implied_vol = call_option_implied_volatility(S0, K_call, T, rf, call_price)
put_implied_vol = put_option_implied_volatility(S0, K_put, T, rf, put_price)

In [64]:
print("Implied Volatilities")
print(f"Call Option Implied Volatility: {round(call_implied_vol, 4)}")
print(f"Put Option Implied Volatility: {round(put_implied_vol, 4)}")

Implied Volatilities
Call Option Implied Volatility: 0.1803
Put Option Implied Volatility: 0.0813


In [65]:
pd.DataFrame(
    {
        "Option Type": ["Call", "Put"],
        "Option Price": [round(call_price, 2), round(put_price, 2)],
        "Implied Volatility": [round(call_implied_vol, 4), round(put_implied_vol, 4)]
    }
)

,Option Type,Option Price,Implied Volatility
0,Call,7.00,0.1803
1,Put,1.06,0.0813


(b) Which options are cheap or expensive?

Based on the model and midpoint of the bid and ask prices, given that the midpoint of call option is 5.38 and model price is 7.08, the call option is cheap. Given that the midpoint of put option is 6.11 and model price is 1.11 (which is concerning regarding model), the put option is expensive.

In [68]:
import numpy as np

S0 = 578.70  
simulations = 10000
N = len(r)

final_prices = np.zeros(simulations)

for sim in range(simulations):
    S_t = S0
    sigma_t_squared = sigma_TGARCH ** 2
    
    for t in range(15):
        Z_t = np.random.normal()
        
        # Calculate the daily return based on current volatility
        daily_return = mu_TGARCH + np.sqrt(sigma_t_squared) * Z_t
        S_t = S_t * np.exp(daily_return)
        
        # Update TGARCH model with simulated return
        a_t = daily_return - mu_TGARCH
        squared_a_t = a_t ** 2
        S_indicator = int(a_t < 0)  # 1 if negative, 0 otherwise

        # Update next day's volatility using TGARCH formula
        sigma_t_squared = abs(
            alpha0_TGARCH +
            alpha1_TGARCH * squared_a_t +
            gamma1 * S_indicator * squared_a_t +
            beta1_TGARCH * sigma_t_squared
        )
    
    # Store the final simulated stock price after 15 days
    final_prices[sim] = S_t

# Calculate option payoffs
call_payoffs_new = np.maximum(final_prices - K_call, 0)
put_payoffs_new = np.maximum(K_put - final_prices, 0)

# Discount payoffs to present value
call_price_new = np.exp(-rf * T) * np.mean(call_payoffs_new)
put_price_new = np.exp(-rf * T) * np.mean(put_payoffs_new)

# Calculate Black-Scholes implied volatilities
call_implied_vol_new = call_option_implied_volatility(S0, K_call, T, rf, call_price_new)
put_implied_vol_new = put_option_implied_volatility(S0, K_put, T, rf, put_price_new)


In [69]:
pd.DataFrame(
    {
        "Option Type": ["Call", "Put"],
        "Option Price": [round(call_price_new, 2), round(put_price_new, 2)],
        "Implied Volatility": [round(call_implied_vol_new, 4), round(put_implied_vol_new, 4)]
    }
)

,Option Type,Option Price,Implied Volatility
0,Call,7.07,0.1840
1,Put,1.03,0.0788


Tje new option prices still suggest the same directionality. The call option is still cheap and the put option is still expensive.

# Problem 3: Stochastiv Volatility Model with Generalized Method of Moments

Consider the following stochastic volatility model:
$$r_t = \mu + \sigma_t \epsilon_t$$
$$\ln{\sigma_t) = \alpha + \phi (\ln{\sigma_{t-1}} - \alpha) + \eta_t$$

where $\epsilon_t \sim N(0,1) $ and $\eta_t \sim N(0, \sigma_\eta^2)$

Let us denote $$\beta^2 = \sigma^2_\eta/(1-\phi^2)$$

Use the following five moment conditions in the GMM:

$$
g_1(\hat{\theta}) = \frac{1}{T} \sum_{t=1}^{T} \left( |r_t| - \mathbb{E}[\sigma_t] \mathbb{E}[|\epsilon_t|] \right)
$$

$$
g_2(\hat{\theta}) = \frac{1}{T} \sum_{t=1}^{T} \left( (r_t)^2 - \mathbb{E}[\sigma_t^2] \mathbb{E}[\epsilon_t^2] \right)
$$

$$
g_3(\hat{\theta}) = \frac{1}{T} \sum_{t=1}^{T} \left( |r_t| |r_{t-1}| - \mathbb{E}[\sigma_t \sigma_{t-1}] \mathbb{E}[|\epsilon_t|] \mathbb{E}[|\epsilon_{t-1}|] \right)
$$

$$
g_4(\hat{\theta}) = \frac{1}{T} \sum_{t=1}^{T} \left( |r_t| |r_{t-2}| - \mathbb{E}[\sigma_t \sigma_{t-2}] \mathbb{E}[|\epsilon_t|] \mathbb{E}[|\epsilon_{t-2}|] \right)
$$

$$
g_5(\hat{\theta}) = \frac{1}{T} \sum_{t=1}^{T} \left( (r_t)^2 (r_{t-1})^2 - \mathbb{E}[\sigma_t^2 \sigma_{t-1}^2] \mathbb{E}[\epsilon_t^2] \mathbb{E}[\epsilon_{t-1}^2] \right)
$$


Let us import the data we are estimating the model on.

In [78]:
ret = pd.read_csv('SV.csv', index_col=0)
ret.head()

,ret
0,0.015336
1,0.000742
2,-0.001326
3,-0.000365
4,-0.008535


Now substitute the parameters into the moment conditions and calculate the objective function.

In [93]:
import numpy as np
from scipy.optimize import minimize

T = len(ret)

initial_params = np.array([0.0, 0.9, 0.2])

E_abs_epsilon = np.sqrt(2 / np.pi)
E_abs_epsilon_sq = 1 

def calculate_expectations(alpha, phi, beta):
    E_sigma = np.exp(alpha + 0.5 * beta**2)
    E_sigma_sq = np.exp(2 * alpha + 2 * beta**2)
    E_sigma_sigma_lag1 = np.exp(2 * alpha + beta**2 * (1 + phi**1))
    E_sigma_sigma_lag2 = np.exp(2 * alpha + beta**2 * (1 + phi**2))
    E_sigma_sq_sigma_sq_lag = np.exp(4 * alpha + 4 * beta**2 * (1 + phi**1))
    
    return E_sigma, E_sigma_sq, E_sigma_sigma_lag1, E_sigma_sigma_lag2, E_sigma_sq_sigma_sq_lag


def moment_conditions(params, returns):
    alpha, phi, beta = params

    # Calculate the expectations based on alpha, phi, and beta
    E_sigma, E_sigma_sq, E_sigma_sigma_lag1, E_sigma_sigma_lag2, E_sigma_sq_sigma_sq_lag = calculate_expectations(alpha, phi, beta)

    # Calculate sample moments based on the data
    abs_returns = np.abs(returns)
    squared_returns = returns**2

    # Moment conditions
    g1 = np.mean(abs_returns) - E_sigma * E_abs_epsilon 
    g2 = np.mean(squared_returns) - E_sigma_sq * E_abs_epsilon_sq 
    g3 = np.mean(abs_returns[1:] * abs_returns[:-1]) - E_sigma_sigma_lag1 * E_abs_epsilon * E_abs_epsilon 
    g4 = np.mean(abs_returns[2:] * abs_returns[:-2]) - E_sigma_sigma_lag2 * E_abs_epsilon * E_abs_epsilon  
    g5 = np.mean(squared_returns[1:] * squared_returns[:-1]) - E_sigma_sq_sigma_sq_lag * E_abs_epsilon_sq * E_abs_epsilon_sq  

    moments = np.array([g1, g2, g3, g4, g5])
    return moments

def gmm_objective(theta, returns):
    moments = moment_conditions(theta, returns)
    objective = np.dot(moments, moments)  # Sum of squared moments
    return objective

result = minimize(gmm_objective, initial_params, args=(ret,), method='BFGS')

theta_estimated = result.x
print("Estimated parameters:", theta_estimated)
print("Objective function value:", result.fun)


Estimated parameters: [-6.55298807  0.53882563  1.72468593]
Objective function value: 9.990616563995977e-07


In [94]:
alpha, phi, beta = theta_estimated

In [95]:
pd.DataFrame(
    {
        "Estimated Parameters": theta_estimated
    },
    index=["alpha", "phi", "beta"]
)

,Estimated Parameters
alpha,-6.552988
phi,0.538826
beta,1.724686
